# RFNSA Antenna Details
##### ©Haris Hassan


In [1]:
################ Import Libraries ################
import sys
#!conda install --yes --prefix {sys.prefix} natsort
from IPython import get_ipython;
#from natsort import index_natsorted
get_ipython().magic('reset -sf')
import pandas as pd
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
import re
import math
import numpy as np
from itertools import count
class text_format:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
# Function to convert from mW to dBm
def W2dBm(mW):
    return 10.*log10(mW) + 30


# Function to convert from dBm to mW
def dBm2W(dBm):
    return 10**((dBm-30)/10.)

################################################

In [2]:
################### Replace with path to your excel file. 
data = pd.read_excel(r'C:\Users\Mewtwo\Downloads\New Microsoft Excel Worksheet.xlsx')
data.fillna(value = 0,inplace = True)

#Height - Phase Centre (m)	Mount Height (m)
MountHeight = []
for l, m in zip(data['Height - Phase Centre (m)'], data['Mount Height (m)']):
    if m == 0:
        MountHeight.append(l)
    else:
        MountHeight.append(m)
data['Height'] = MountHeight
#remove the proposed Antennas
data = data.drop(data[data['Existing/Proposed'] == 'Proposed'].index)
data = data.reset_index(drop=True)

In [3]:
################ Create the dataframe from meaningful columns of STAD table and rename the columns ################
df = pd.DataFrame(data, columns=['Antenna ID No','Add ID','Antenna Model','Sector','Height'
                                 ,'Bearing Degrees (true)',
                                 'Mech Downtilt','Elect Downtilt','System',
                                 'Port Number (Band Power per Port (dBm))','Band Power per Port (dBm)','Notes'])
df = df.rename(columns={'Antenna Model': 'Antenna', 'Antenna ID No': 'ID', 'Bearing Degrees (true)': 'Bearing'
                  , 'Mech Downtilt': 'MDT', 'Elect Downtilt': 'EDT', 
                   'Port Number (Band Power per Port (dBm))': 'Possible Ports', 
                   'Band Power per Port (dBm)': 'Powers'})

df.fillna(value = 0,inplace = True)
df = df.reset_index(drop=True)


In [4]:
#dfSiteInfo = pd.DataFrame(data, columns=['Structure Owner','Latitude', 'Longitude', 'Region', 'Structure',
#                                         'Site System (Structure No)','NSA','Site Name'])
dfSiteInfo = pd.DataFrame(data, columns=['NSA','Latitude', 'Longitude', 'Region'])
dfSiteInfo['Coordinates'] = dfSiteInfo.apply(lambda row: str(row['Latitude']) + ', ' + str(row['Longitude']), axis=1)
dfSiteInfo = dfSiteInfo.drop('Latitude', axis=1)
dfSiteInfo = dfSiteInfo.drop('Longitude', axis=1)

dfStructureInfo = pd.DataFrame(data, columns=['Structure Owner','Structure',
                                         'Site System (Structure No)',])
dfSiteNames = pd.DataFrame(data, columns=['System','Site Name'])
dfSiteNames['Operator'] = dfSiteNames['System'].str.split(" /").str[0]
dfSiteNames = dfSiteNames.drop('System', axis=1)
dfSiteNames['Site Name']=dfSiteNames['Site Name'].apply(lambda x: x.title())

dfStructureInfo['Structure'] = ['Structure ' +f'{l}' + ': ' + ''.join(''.join(map(str, m))) for l, m in zip(dfStructureInfo['Site System (Structure No)'],dfStructureInfo['Structure'])]
dfStructureInfo = dfStructureInfo.drop('Site System (Structure No)', axis=1)


dfSiteNames = dfSiteNames.drop_duplicates(keep='first')
dfSiteInfo = dfSiteInfo.drop_duplicates(keep='first')
dfStructureInfo = dfStructureInfo.drop_duplicates(keep='first')

display(dfSiteInfo)
display(dfSiteNames)
display(dfStructureInfo)

,NSA,Region,Coordinates
0,2795001,NSW,"-33.45006, 149.547"


,Site Name,Operator
0,Bathurst,Optus
38,Bathurst,Vodafone
62,Mt. Panorama / Wahluu,NSW Government - Telco Authority
65,Mt Panorama,NBN Co


,Structure Owner,Structure
0,Axicom,Structure 1: 47.8m Lattice Tower


In [5]:
AntennasUnique = list( dict.fromkeys(df['Antenna']) )
#AntennasUnique = sorted(AntennasUnique, key=lambda x: int(re.search('\d+', x).group(0)))
#display(AntennasUnique)
IDUnique = list( dict.fromkeys(df['ID'],df['Antenna']) )
#Special and Unique function to sort the ids string universally
#IDUnique = sorted(IDUnique, key=lambda x: (2,int(x)) if re.search('^\d+$', x) else (1,int(re.search('\d+', x).group(0))) if re.search('\d+$', x) else (0, x))

In [6]:
# Extracting Carrier (optus, telstra, Vodafone), technology/Frequency (i.e LTE900, NR2100, WCDMA850) information from System 

df['System'] = df['System'].str.replace(' - LOCKED', '', regex=True)
df['System'] = df['System'].str.replace('[Macro]', '', regex=False)
df['System'] = df['System'].str.replace(' \[', ';', regex=True)
df['System'] = df['System'].str.replace('/', ';', 1, regex=True)
df['Operator'] = df['System'].str.split(" ;").str[0]

In [7]:
#display(df.loc[(df['Antenna'] == 'RV4PX310R') &  (df['ID'] == '21-J'), {'ID','MDT'}])

In [8]:
Carriers=[]
for idx in IDUnique:
    multicarrier = []
    counter = 0
    #display(df.loc[df['ID'] == idx, {'Antenna','System','Carrier','Tech'}])
    for x in df.loc[df['ID'] == idx, 'Operator']:
        multicarrier.append(x)
        counter +=1
        #display(idx)
        #display(x)
    for c in range(counter):
        Carriers.append('/'.join(list( dict.fromkeys(multicarrier))))
df['Carrier'] = Carriers

In [9]:
#df['Tech'] = df['System'].str.split(";").str[1]
tech = []
df['Tech'] = df['System'].str.split(";").str[1]
for x, y in enumerate(df['Carrier']):
    if '/' in y:
        tech.append(''.join(df['System'][x].split(";")))
    else:
        tech.append(df['System'][x].split(";")[1])
        
df['Tech'] = tech

In [10]:

df["sorter"] =  df["Carrier"] + df["ID"].astype(str)
df = df.iloc[df['sorter'].astype(str).apply(lambda x: (2, int(x)) if re.search('^\d+$', x) else (1, int(re.search('\d+', x).group(0))) if re.search('\d+$', x) else (0, x)).argsort()]
df = df.reset_index(drop=True)
df.index += 1

AntennasUnique = list( dict.fromkeys(df['Antenna']) )
#AntennasUnique = sorted(AntennasUnique, key=lambda x: int(re.search('\d+', x).group(0)))
#display(AntennasUnique)
IDUnique = list( dict.fromkeys(df['ID'],df['Antenna']) )
#Special and Unique function to sort the ids string universally
#IDUnique = sorted(IDUnique, key=lambda x: (2,int(x)) if re.search('^\d+$', x) else (1,int(re.search('\d+', x).group(0))) if re.search('\d+$', x) else (0, x))
AntennaIds = {}
for antenna in AntennasUnique:
    IdsForAntenna = list(dict.fromkeys(df.loc[df['Antenna'] == antenna, 'ID']))
    AntennaIds[antenna] = IdsForAntenna
    
#df = df.iloc[index_natsorted(df.sorter)]
# Sorting the STAD dataframe to match with RFNSA
#df.index = df['sorter'].str.replace('[a-zA-Z]+','.').astype(float)
#df = df.sort_index().reset_index(drop=True)
#df.sort_values('sorter',key=lambda x: (2,int(x)) if re.search('^\d+$', x) else (1,int(re.search('\d+', x).group(0))) if re.search('\d+$', x) else (0, x))
#IDUnique = sorted(IDUnique, key=lambda x: (2,int(x)) if re.search('^\d+$', x) else (1,int(re.search('\d+', x).group(0))) if re.search('\d+$', x) else (0, x))
#df.sort_values(by=['sorter'], ascending=[True], inplace = True)
#df.sort_values(by=['Carrier','ID'], ascending=[True, True], inplace = True)
#Reset index of dataframe after sorting and start from 1


#Count the total ports of each antenna
df['Total Ports'] = df['Powers'].str.count(';')
df.fillna(value = 0,inplace = True)
df['Total Ports'] = df['Total Ports'] + 1
df['Total Ports'] = list(x for x in df['Total Ports'])

#Format/Clean the Data
df['EDT'] = df['EDT'].str.replace(' to ', '-', regex=True)
df['EDT'] = df['EDT'].str.replace(r'\(.*\)','', regex=True)
df['Sector'] = df['Sector'].astype(str).replace('\.0', '', regex=True)
df['System/Sector']= [''.join(''.join(map(str, m))+' - Sector ' +f'{l}') for l, m in zip(df['Sector'],df['Tech'])]

#Correcting microwavelinks
templist = []
for x in df['Powers']:
    if ';' not in str(x):
        templist.append(str(x)+';')
    else:
        templist.append(str(x))        
df['Powers'] = templist

In [11]:
# Find the sectors
Sectors = []
for idx in IDUnique:
    Sectorstemp = []
    counter = 0
    for x in df.loc[df['ID'] == idx, 'Sector']:
        Sectorstemp.append(str(x))
        counter +=1
        #display(idx)
    for c in range(counter):
        Sectors.append(', '.join(list( dict.fromkeys(Sectorstemp))))
df['Sectors'] = Sectors

In [12]:
### Convert te Power(dBm) into Watts
temp_power = []
for elm in df['Powers'].str.split(";"):
    if isinstance(elm, (list, tuple)):
        elm = [x for x in elm if x != '']  # Remove any empty strings from the list
        converted_values = []
        for sub in elm:
            if float(sub) >= 30:
                try:
                    converted_values.append(round(dBm2W(float(sub)),1))
                except ValueError:
                    converted_values.append(sub)
                    pass  # Skip the current value if it cannot be converted to a float
            else:
                 try:
                    converted_values.append(round(dBm2W(float(sub)),4))
                 except ValueError:
                    converted_values.append(sub)
                    pass  # Skip the current value if it cannot be converted to a float
        temp_power.append(converted_values)
    else:
        try:
            temp_power.append([dBm2W(float(elm))])
        except ValueError:
            temp_power.append([])

df['Powers (W)'] = temp_power

In [13]:
display(df)

,ID,Add ID,Antenna,Sector,Height,Bearing,MDT,EDT,System,Possible Ports,Powers,Notes,Operator,Carrier,Tech,sorter,Total Ports,System/Sector,Sectors,Powers (W)
1,A,2BTH-51-08-MPRP,UKY 220 28/DC15,0,38.85,208,0,0,NBN Co ; Microwave Link,0,25;25,NBN Co. Microwave Link Configuration(R01),NBN Co,NBN Co,Microwave Link,NBN CoA,2.0,Microwave Link - Sector 0,0,"[0.3162, 0.3162]"
2,GRNA01L,201251,BA80-67 2X4,0,48.60,0,0,0,NSW Government - Telco Authority ; Gov. Radio Network,0,48.82;,0,NSW Government - Telco Authority,NSW Government - Telco Authority,Gov. Radio Network,NSW Government - Telco AuthorityGRNA01L,1.0,Gov. Radio Network - Sector 0,0,[76.2]
3,GRNA01U,201251,BA80-67 2X4,0,51.60,0,0,0,NSW Government - Telco Authority ; Gov. Radio Network,0,0;,0,NSW Government - Telco Authority,NSW Government - Telco Authority,Gov. Radio Network,NSW Government - Telco AuthorityGRNA01U,1.0,Gov. Radio Network - Sector 0,0,[0.001]
4,11-O,S0120,R2V4PX310R,1,46.61,0,0,0-10,Optus ; WCDMA2100,5;6;7;8;9;10;11;12,;;;;43;;;;;;;,"OPTUS RANCAP: TU, RFU Solution, FY21 power config;",Optus,Optus,WCDMA2100,Optus11-O,12.0,WCDMA2100 - Sector 1,1,[20.0]
5,11-O,S0120,R2V4PX310R,1,46.61,0,0,0-10,Optus ; WCDMA900,1;2;3;4,;;46;;;;;;;;;,"OPTUS RANCAP: U900, RFU Solution, FY21 power config;",Optus,Optus,WCDMA900,Optus11-O,12.0,WCDMA900 - Sector 1,1,[39.8]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,3,2BTH-51-08-MPRP,LLPX310R,3,43.50,170,0,0-10,NBN Co ; LTE2300,1;2;3;4,46;46;;,NBN Co. LTE2300 Configuration(R01),NBN Co,NBN Co,LTE2300,NBN Co3,4.0,LTE2300 - Sector 3,3,"[39.8, 39.8]"
74,4,2BTH-51-08-MPRP,LLPX210R-V1,4,43.50,80,0,0-10,NBN Co ; LTE2300,1;2;3;4,46;46;;,NBN Co. LTE2300 Configuration(R01),NBN Co,NBN Co,LTE2300,NBN Co4,4.0,LTE2300 - Sector 4,4,"[39.8, 39.8]"
75,5,2BTH-51-08-MPRP,LLPX310R,5,43.50,130,0,0-10,NBN Co ; LTE2300,1;2;3;4,46;46;;,NBN Co. LTE2300 Configuration(R01),NBN Co,NBN Co,LTE2300,NBN Co5,4.0,LTE2300 - Sector 5,5,"[39.8, 39.8]"
76,5,2BTH-51-08-MPRP,LLPX310R,5,43.50,130,0,0-10,NBN Co ; LTE2300,1;2;3;4,;;46;46,NBN Co. LTE2300 Configuration(R01),NBN Co,NBN Co,LTE2300,NBN Co5,4.0,LTE2300 - Sector 5,5,"[39.8, 39.8]"


In [14]:
######## what should be the frequency? ########
Assess_Freq_list = []
for i in df['Tech']:
    if any([x in i for x in ['GSM900', 'WCDMA850', 'NB-IOT900', 'WCDMA900','LTE850', 'NR850','LTE900']]):
        Assess_Freq_list.append('900')
    elif any([x in i for x in ['NR/LTE2100', 'WCDMA2100', 'LTE2100']]):
        Assess_Freq_list.append('2100')
    elif any([x in i for x in ['NR/LTE1800', 'LTE1800']]):
        Assess_Freq_list.append('1800')
    elif any([x in i for x in ['LTE2600', 'NR2600']]):
        Assess_Freq_list.append('2600')
    elif 'LTE700' in i:
        Assess_Freq_list.append('750')
    elif 'LTE2300' in i:
        Assess_Freq_list.append('2350')
    else:
        Assess_Freq_list.append(i)
df['Assess Freq'] = Assess_Freq_list

In [15]:
######## Which ports to add powers to? ########
Mylist =[]

for i in df['Powers']:
    count = 1
    lister = []
    if isinstance(i, str):
        for j in i:
            if j==';':
                count+=1
            else:
                lister.append(count)
        Mylist.append(lister)
    else:
        Mylist.append(list(str(i))) 
        
    
new_k = []
for elem in Mylist:
    kiterator=[]
    for el in elem:
        if el not in kiterator:
            kiterator.append(el)
    new_k.append(kiterator)
k = new_k

df['WhereToAddPower'] = k

In [16]:
####Formatting Powers Column with ports numbers

#itArr=[]
#for index, elm in df['Powers'].iteritems():
#    if isinstance(elm, (list, tuple)):
#        itArr.append([f'{i+1}. {l}' for i, l in enumerate(elm)])
#    else:
#        itArr.append([elm])
#df=df.drop(columns='Powers', axis=1)
#df.insert(2, 'Powers', itArr)
#############################################

## Add these Antennas to prox5

In [17]:
print('You need to add these antennas\n\n'+str.join(" \n", AntennasUnique)+' \n\nto PROX5')

You need to add these antennas

UKY 220 28/DC15 
BA80-67 2X4 
R2V4PX310R 
AIR6488 B78H 
6W22ME-01 
TTB-609017/172717/172717DER-60F(B) 
SSPX310R 
LLPX310R 
BA40-41 2X2 
LLPX210R-V1 

to PROX5


## IDs for Antennas

In [18]:
for x, y in AntennaIds.items():
    print(f'\nThe id of '+text_format.BOLD+ x + text_format.END + ' are\n'+str.join("", str(y)))



The id of UKY 220 28/DC15 are
['A']

The id of BA80-67 2X4 are
['GRNA01L', 'GRNA01U']

The id of R2V4PX310R are
['11-O', '21-O', '31-O', '41-O']

The id of AIR6488 B78H are
['12-O', '22-O', '32-O', '42-O']

The id of 6W22ME-01 are
['23-O']

The id of TTB-609017/172717/172717DER-60F(B) are
['11-V', '12-V', '21-V', '22-V', '31-V', '32-V']

The id of SSPX310R are
['B1']

The id of LLPX310R are
[1, 3, 5, 6]

The id of BA40-41 2X2 are
['GRNA02']

The id of LLPX210R-V1 are
[2, 4]


## Antenna Settings

In [19]:
display('Set Power and Frequency to ports in Prox5')
Settings = {}
#for i in IDUnique:
#    IdsForAntenna = list(dict.fromkeys(df.loc[df['ID'] == i, 'ID']))
#    AntennaIds[antenna] = IdsForAntenna

for i in IDUnique:
        print(f'\nFor '+text_format.BOLD+str(i)+text_format.END+f', the Settings are')
        display(df.loc[(df['ID'] == i),['ID','Height','Bearing','MDT']])

'Set Power and Frequency to ports in Prox5'


For A, the Settings are


,ID,Height,Bearing,MDT
1,A,38.85,208,0



For GRNA01L, the Settings are


,ID,Height,Bearing,MDT
2,GRNA01L,48.6,0,0


...

,ID,Height,Bearing,MDT
74,4,43.5,80,0



For 5, the Settings are


,ID,Height,Bearing,MDT
75,5,43.5,130,0
76,5,43.5,130,0



For 6, the Settings are


,ID,Height,Bearing,MDT
77,6,43.5,260,0


## Set Power and Frequency to ports in Prox5

In [20]:
display('Set Power and Frequency to ports in Prox5')
for i in IDUnique:
        print(text_format.BOLD+str(i))
        display(df.loc[(df['ID'] == i),['ID','EDT','Tech','Possible Ports','WhereToAddPower','Powers (W)']])

'Set Power and Frequency to ports in Prox5'

A


,ID,EDT,Tech,Possible Ports,WhereToAddPower,Powers (W)
1,A,0,Microwave Link,0,"[1, 2]","[0.3162, 0.3162]"


GRNA01L


,ID,EDT,Tech,Possible Ports,WhereToAddPower,Powers (W)
2,GRNA01L,0,Gov. Radio Network,0,[1],[76.2]


...

,ID,EDT,Tech,Possible Ports,WhereToAddPower,Powers (W)
74,4,0-10,LTE2300,1;2;3;4,"[1, 2]","[39.8, 39.8]"


5


,ID,EDT,Tech,Possible Ports,WhereToAddPower,Powers (W)
75,5,0-10,LTE2300,1;2;3;4,"[1, 2]","[39.8, 39.8]"
76,5,0-10,LTE2300,1;2;3;4,"[3, 4]","[39.8, 39.8]"


6


,ID,EDT,Tech,Possible Ports,WhereToAddPower,Powers (W)
77,6,0-10,LTE2300,1;2;3;4,"[1, 2]","[39.8, 39.8]"


## EMEG Equipment List

In [21]:

totalPortsdict = {}
for idantenna in IDUnique:
    totalPorts = list(dict.fromkeys(df.loc[df['ID'] == idantenna, 'Total Ports']))
    totalPortsdict[idantenna] = totalPorts
    
#display(totalPortsdict)   
#print(df.loc[(df['ID']=='11-O'), ['EDT','Sector','WhereToAddPower','Power (W)']])
#display(df.loc[(df['ID'] == i),['ID','Height','Bearing','MDT']])

listpow = []
listsec = []
x = 0
yprev = 0
for AntId, AntPorts in totalPortsdict.items():
    templistpow = [''] * (int(AntPorts[0] / 2) + int(AntPorts[0] % 2))
    templistsec = [''] * (int(AntPorts[0] / 2) + int(AntPorts[0] % 2))
    tempdf = df.loc[(df['ID']==AntId), ['EDT','System/Sector','WhereToAddPower','Powers (W)']]
    for sector, port, power in zip(tempdf['System/Sector'], tempdf['WhereToAddPower'], tempdf['Powers (W)']):
        display(AntId)
        if len(port)>1:
            for x, y in enumerate(port):       
                if sector not in templistsec[math.ceil(int(y)/2)-1]:
                    templistsec[math.ceil(int(y)/2)-1] += sector +'\n'
                    templistpow[math.ceil(int(y)/2)-1] += '\n'
                templistpow[math.ceil(int(y)/2)-1] += '+' + str(power[x])
        if len(port)==1:
            for x, y in enumerate(port):
                display(math.ceil(int(y)/2)-1)
                templistsec[math.ceil(int(y)/2)-1] += sector +'\n'
                if y%2==0:
                    templistpow[math.ceil(int(y)/2)-1] += '\n0++' + str(power[x])
                else:
                    templistpow[math.ceil(int(y)/2)-1] += '\n' + str(power[x]) + '++0'
            
    listpow.append(templistpow)
    listsec.append(templistsec)

for index1, x in enumerate(listpow):
    for index2, y in enumerate(x):
        if y == '':
            listpow[index1][index2] = '0'
        
for index1, x in enumerate(listsec):
    for index2, y in enumerate(x):
        if y == '':
            listsec[index1][index2] = '-'
        

EquipmentList = pd.DataFrame(totalPortsdict.items(), columns=['ID', 'No. of ports'])
EquipmentList['System/Sector'] = listsec
EquipmentList['Power (W)'] = listpow

EquipmentList_final = pd.DataFrame([], columns=['ID','System/Sector', 'Power (W)'])

for a, x, y in zip(EquipmentList['ID'],EquipmentList['System/Sector'], EquipmentList['Power (W)']): 
        EquipmentListAntenna = pd.DataFrame([a]*len(x),columns=['ID'])
        EquipmentListAntenna['System/Sector'] = x
        EquipmentListAntenna['Power (W)'] = y
        EquipmentList_final = pd.concat([EquipmentList_final, EquipmentListAntenna], axis=0)

EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.replace('\+', '', 1, regex=True)
EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.replace('\+\+', '+', regex=True)
EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.replace('\\n\+', '\\n', regex=True)
EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.replace('+', ' + ', regex=False)
EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.strip()

EquipmentList_final['System/Sector'] = EquipmentList_final['System/Sector'].str.replace('\n ', '\n', regex=False)
EquipmentList_final['System/Sector'] = EquipmentList_final['System/Sector'].str.strip()
#display(EquipmentList_final)


'A'

'GRNA01L'

0

'GRNA01U'

0

...

3

4

5

5

6

## CANRAD

In [22]:
listpow = []
listfunc = []
listport = []
x = 0
yprev = 0

for AntId, AntPorts in totalPortsdict.items():
    templistpow = [0] * int(AntPorts[0])
    templistfunc = [''] * int(AntPorts[0])
    templistport = list(range(1,int(AntPorts[0])+1))
    #tempfunc = [''] * int(AntPorts[0])
    tempdf = df.loc[(df['ID']==AntId), ['Sector','Tech','WhereToAddPower','Powers (W)']]
    for sector, tech, port, power in zip(tempdf['Sector'], tempdf['Tech'], tempdf['WhereToAddPower'], tempdf['Powers (W)']):
        for x, y in enumerate(port):       
            #if sector not in templistfunc[y]:
            templistfunc[y-1] += tech +'/'
            templistpow[y-1] = str(round(float(templistpow[y-1]) + float(power[x]),3))
    listpow.append(templistpow)
    listport.append(templistport)
    listfunc.append(templistfunc)

CANRAD = pd.DataFrame(totalPortsdict.items(), columns=['ID', 'No. of ports'])
CANRAD['Port'] = listport
CANRAD['Function'] = listfunc
CANRAD['Total Power'] = listpow

CANRAD_final = pd.DataFrame([], columns=['ID','Function', 'Total Power'])

for a, z, x, y in zip(CANRAD['ID'], CANRAD['Port'],CANRAD['Function'], CANRAD['Total Power']): 
        CANRADAntenna = pd.DataFrame([a]*len(x),columns=['ID'])
        CANRADAntenna['Port'] = z
        CANRADAntenna['Function'] = x
        CANRADAntenna['Total Power'] = y
        CANRAD_final = pd.concat([CANRAD_final, CANRADAntenna], axis=0)
CANRAD_final=CANRAD_final.iloc[:, [0,3,1,2]]
display(CANRAD_final)


,ID,Port,Function,Total Power
0,A,1.0,Microwave Link/,0.316
1,A,2.0,Microwave Link/,0.316
0,GRNA01L,1.0,Gov. Radio Network/,76.2
0,GRNA01U,1.0,Gov. Radio Network/,0.001
0,11-O,1.0,LTE700 /,63.1
...,...,...,...,...
3,5,4.0,LTE2300 /,39.8
0,6,1.0,LTE2300 /,39.8
1,6,2.0,LTE2300 /,39.8
2,6,3.0,,0


## Exports

In [23]:
#df3 = pd.DataFrame.from_dict(AntennaIds)
df3 = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in AntennaIds.items() ]))
df3 = df3.fillna('')



dfAntennaSettings = df[['Carrier','ID','Height','Bearing','MDT','Total Ports','Sectors']].copy()
dfAntennaSettings = dfAntennaSettings.drop_duplicates(keep='first')

path = r'C:\Users\Mewtwo\Desktop'+'\\'+ str(data['NSA'][3])+ ' ' + data['Site Name'][3] + ' RFNSA Wrangled Data.xlsx'

from openpyxl.utils import get_column_letter
from openpyxl import Workbook
from openpyxl import load_workbook

wb = Workbook()
wb.save(path)

writer = pd.ExcelWriter(path, engine='openpyxl')
df3.to_excel(writer, sheet_name = 'Antenna IDs', index=False)
dfAntennaSettings.to_excel(writer, sheet_name = 'Antenna Settings', index=False)
df.to_excel(writer, sheet_name = 'Add powers to Prox', columns = ['Antenna','ID','EDT','Tech','Possible Ports','WhereToAddPower','Powers (W)','Assess Freq','Notes'], index=False)
EquipmentList_final.to_excel(writer, sheet_name = 'EMEG List', index=False)
CANRAD_final.to_excel(writer, sheet_name = 'CANRAD', index=False)
dfSiteInfo.to_excel(writer, sheet_name = 'Site Info', index=False)
dfSiteNames.to_excel(writer, sheet_name = 'Site Info', startcol = 3,index=False)
dfStructureInfo.to_excel(writer, sheet_name = 'Site Info',startcol = 5, index=False)


# load the Excel file with openpyxl
workbook = writer.book

for sheet in workbook:
    for column in range(1, sheet.max_column + 2):
        letter = get_column_letter(column)
        #sheet.column_dimensions[letter].auto_size = True
        sheet.column_dimensions[letter].bestFit = True

# save the modified workbook
workbook.save(path)

In [24]:
#display( HTML( df.to_html().replace("\\n","<br><li type='1'>") ) )
EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.replace(r'\n','<br>', regex=True)
#EquipmentList_final['System/Sector'] = EquipmentList_final['System/Sector'].str.replace((r'\n','<br>', regex=True)

EquipmentList_final['System/Sector'] = EquipmentList_final['System/Sector'].str.replace(r'\n','&#10;')

#EquipmentList_final.to_html(r'C:\Users\Mewtwo\Desktop\EMEGList.html', index=False)
#dfAntennaSettings.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata.html', index=False)
#df3.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata2.html',index=False)
#df.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata3.html', columns = ['ID','EDT','Tech','Possible Ports','WhereToAddPower','Powers (W)','Assess Freq','Notes'], index=False)

#df.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata4.html',columns = ['ID','EDT','Sector','WhereToAddPower','Powers (W)'],index=False)
#import subprocess
#subprocess.call('wkhtmltoimage -f png --width 0 Antennadata.html Antennadata.png', shell=True)


C:\Users\Mewtwo\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [25]:

for sheetname in workbook.sheetnames:
    worksheet = workbook[sheetname]

    # set best fit attribute to true for all columns
    for column_cells in worksheet.columns:
        length = max(len(str(cell.value)) for cell in column_cells)
        worksheet.column_dimensions[column_cells[0].column_letter].width = length + 5
        worksheet.column_dimensions[column_cells[0].column_letter].bestFit = True

worksheet = workbook.active

# set the width of column A to 15
column_letter = get_column_letter(1)  # A
column_dimension = worksheet.column_dimensions[column_letter]
column_dimension.width = 15
column_dimension.bestFit = True


In [26]:
#with open("C:\\Users\\Mewtwo\\Downloads\\RFNSA Scrapper_V2.html", encoding="utf8") as html_file:
#    content = html_file.read()

# Get rid off prompts and source code
#content = content.replace("div.input_area {","div.input_area {\n\tdisplay: none;")    
#content = content.replace(".prompt {",".prompt {\n\tdisplay: none;")

#f = open(FILE, 'w')
#f.write(content)
#f.close()

In [27]:
#jupyter nbconvert --to pdf 'RFNSA Scrapper_V2.ipynb' --template=hidecode.tplx
